In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tempfile
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder


from tensorflow.contrib.tensor_forest.client import eval_metrics
from tensorflow.contrib.tensor_forest.client import random_forest
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.estimator.inputs import numpy_io
from tensorflow.contrib.learn.python.learn import metric_spec

In [2]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns 

    def fit(self,X,y=None):
        return self 

    def transform(self,X):
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [3]:
data_reg = pd.DataFrame.from_csv('mushrooms.csv', header=0, index_col=None)
data = MultiColumnLabelEncoder(columns = data_reg.columns.values).fit_transform(data_reg) #label data in columns

c:\users\domen\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [4]:
data = data.values
N = len(data) 
np.random.shuffle(data) #shuffle rows

training = data[:int(N*0.8)] #split training and test data
test = data[int(N*0.8):]

training_x = training[:,1:] 
training_y = training[:,0] 
training_x = normalize(training_x, axis=0, norm='max')

test_x = test[:,1:]
test_y = test[:,0] 
test_x = normalize(test_x, axis=0, norm='max')

In [5]:
def build_estimator(model_dir):
  """Build an estimator."""
  params = tensor_forest.ForestHParams(
      num_classes=2,
      num_features=22,
      num_trees=100,
      max_nodes=1000)
  graph_builder_class = tensor_forest.RandomForestGraphs
  return random_forest.TensorForestEstimator(
      params, graph_builder_class=graph_builder_class, model_dir=model_dir)


def train_and_eval(training_x,training_y,test_x,test_y):
  """Train and evaluate the model."""
  model_dir = tempfile.mkdtemp()
  print('model directory = %s' % model_dir)

  est = build_estimator(model_dir)

  train_input_fn = numpy_io.numpy_input_fn(
      x={'mushrooms': training_x.view('float32')},
      y=training_y.view('int'),
      batch_size=200,
      num_epochs=None,
      shuffle=True)
  est.fit(input_fn=train_input_fn, steps=None)
    
   

  metric_name = 'accuracy'
  metric = {
      metric_name:
          metric_spec.MetricSpec(
              eval_metrics.get_metric(metric_name),
              prediction_key=eval_metrics.get_prediction_key(metric_name))
  }

  test_input_fn = numpy_io.numpy_input_fn(
      x={'mushrooms': test_x.view('float32')},
      y=test_y.view('int'),
      num_epochs=1,
      batch_size=200,
      shuffle=False)

  results = est.evaluate(input_fn=test_input_fn, metrics=metric)
  for key in sorted(results):
    print('%s: %s' % (key, results[key]))
  


train_and_eval(training_x,training_y,test_x,test_y)

model directory = C:\Users\Domen\AppData\Local\Temp\tmp7jd1a90b
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Domen\\AppData\\Local\\Temp\\tmp7jd1a90b', '_save_checkpoints_steps': None, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_evaluation_master': '', '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001942057A898>, '_task_id': 0, '_num_worker_replicas': 0, '_task_type': None, '_save_checkpoints_secs': 600, '_environment': 'local', '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_ps_replicas': 0, '_master': ''}
INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'bagged_features': None, 'bagging_fraction': 1.0, 'min_split_samples': 5, 'base_random_seed': 0, 'max_nodes': 1000, 'num_splits_to_consider': 22, 'regression': False, 'dominate_fract

c:\users\domen\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead


INFO:tensorflow:dense_features_size: 22 dense: [{name: mushrooms original_type: 0 size: 22}] sparse: []
INFO:tensorflow:training graph for tree: 0
INFO:tensorflow:training graph for tree: 1
INFO:tensorflow:training graph for tree: 2
INFO:tensorflow:training graph for tree: 3
INFO:tensorflow:training graph for tree: 4
INFO:tensorflow:training graph for tree: 5
INFO:tensorflow:training graph for tree: 6
INFO:tensorflow:training graph for tree: 7
INFO:tensorflow:training graph for tree: 8
INFO:tensorflow:training graph for tree: 9
INFO:tensorflow:training graph for tree: 10
INFO:tensorflow:training graph for tree: 11
INFO:tensorflow:training graph for tree: 12
INFO:tensorflow:training graph for tree: 13
INFO:tensorflow:training graph for tree: 14
INFO:tensorflow:training graph for tree: 15
INFO:tensorflow:training graph for tree: 16
INFO:tensorflow:training graph for tree: 17
INFO:tensorflow:training graph for tree: 18
INFO:tensorflow:training graph for tree: 19
INFO:tensorflow:training g

c:\users\domen\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead


INFO:tensorflow:dense_features_size: 22 dense: [{name: mushrooms original_type: 0 size: 22}] sparse: []
INFO:tensorflow:Starting evaluation at 2017-12-31-13:26:01
INFO:tensorflow:Restoring parameters from C:\Users\Domen\AppData\Local\Temp\tmp7jd1a90b\model.ckpt-1003
INFO:tensorflow:Finished evaluation at 2017-12-31-13:26:11
INFO:tensorflow:Saving dict for global step 1003: accuracy = 0.962769, global_step = 1003, loss = -998.0
accuracy: 0.962769
global_step: 1003
loss: -998.0
